# Types of routing

## Map Initialisation

In [22]:
import folium
import folium.plugins as plugins

# Center coordinates for the map
center_coordinates = [53.36486137451511, -1.8160056925378616]  # Edale

tiles = "https://api.os.uk/maps/raster/v1/zxy/Leisure_27700/{z}/{x}/{y}.png?key=GGF16a7QH0WAQEkXEXqCwHwxLp0aoMAL"
tiles_mapbox = "https://api.mapbox.com/styles/v1/mapbox/outdoors-v12/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZGFuLWxlZTc2IiwiYSI6ImNsaXJ4d3N3azE3M3Mza28xdnVmdGxwczcifQ.j0THIt59Yt7D6qG1WKXTPg"
tiles_mapbox2 = "https://api.mapbox.com/v4/dan-lee76.Outdoors/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoiZGFuLWxlZTc2IiwiYSI6ImNsaXJ4d3N3azE3M3Mza28xdnVmdGxwczcifQ.j0THIt59Yt7D6qG1WKXTPg"
tile_backup = "opentopomap"

# Initialize the map using OS Maps as the default tiles
m = folium.Map(location=center_coordinates, zoom_start=13, max_zoom=20, min_zoom=0, attr="© Ordnance Survey")

folium.TileLayer(
    name="Mapbox Outdoors",
    tiles="https://api.mapbox.com/styles/v1/mapbox/outdoors-v12/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZGFuLWxlZTc2IiwiYSI6ImNsaXJ4d3N3azE3M3Mza28xdnVmdGxwczcifQ.j0THIt59Yt7D6qG1WKXTPg",
    attr="© Mapbox",
    overlay=False,
    control=True,
    show=False,
).add_to(m)

folium.TileLayer(
    name="OS Maps Road",
    tiles="https://api.os.uk/maps/raster/v1/zxy/Outdoor_3857/{z}/{x}/{y}.png?key=GGF16a7QH0WAQEkXEXqCwHwxLp0aoMAL",
    attr="© Ordnance Survey",
    overlay=False,
    control=True,
    show=False,
).add_to(m)

# folium.TileLayer(
#     name="OS Maps Lesiure",
#     tiles="https://api.os.uk/maps/raster/v1/zxy/Leisure_27700/{z}/{x}/{y}.png?key=GGF16a7QH0WAQEkXEXqCwHwxLp0aoMAL",
#     attr="© Ordnance Survey",
#     overlay=False,
#     control=True,
#     crs="EPSG27700",
# ).add_to(m)

folium.TileLayer(
    name="OpenTopoMap",
    tiles="https://a.tile.opentopomap.org/{z}/{x}/{y}.png",
    attr="© OpenTopoMap",
    overlay=False,
    control=True,
    show=True,
).add_to(m)

folium.GeoJson(
    data="route_buffer.geojson",
    name="GeoJSON",
    overlay=True,
    control=True,
    show=False,
).add_to(m)

folium.plugins.ScrollZoomToggler().add_to(m)

folium.LayerControl().add_to(m)

# Circular

## GA

### Code

In [1]:
import GA
coords = GA.generate_route()

KeyboardInterrupt: 

In [3]:
folium.PolyLine(
    locations=coords,
    color="#FF0000",
    weight=5,
    tooltip="GA",
    opacity=0.5,
).add_to(m)

### Outcome

In [4]:
m

# Point-to-point routing


## A-Star with arcs

1. Creates an arc between points A and B, of the target distance (a star between the points)
2. Create a buffer around the arc, currently target distance * 2.5%
3. For each intersection within the boundary, calculate a route between them A -> b -> B
4. Pick closest route to distance

Time: <7 Seconds (All Routes)

Pros: Quick, consistent results
Cons: Doesn't always work (specific cases)

Improvement -> Use Govs open data on green areas, and look at elevation, to create a heuristic which will rank the routes. 
Instead of using the boundary, can iterate through each point

In [23]:
import a_star_random_with_arcs as AStarArcs
gen = AStarArcs.AStarArcs(10, 1)
gen.main()

In [25]:
coords = gen.generate_new()
folium.PolyLine(
    locations=coords,
    color="#00FF00",
    weight=5,
    tooltip="P2P-A*",
    opacity=0.5,
).add_to(m)
m

Distance Achieved: 9.025536000000002
Time: 947.6572983264923


## Genetic/Memetic Algorithm

1. Generate population (100) which meets the criteria of distance. (Greedy search)
2. Complete generations (50)
   1. Selection - using roulette. Primarily using SoftMax function. Which converts them into raw probabilities based on the population
   2. Crossover - Pick a random point on both parents, and breed them
   3. Mutation - 20% mutation rate, where it does random extensions within
   4. Local Search(*) - *Picks a point, moves to a different neighbour, then extend route, avoiding current path*
3. Filter best route
4. Fitness - Based on distance from end node to target node, and length of route

Time: 2 mins for best route, based on 100/100
Pros: Can generate *some* decent routes. Still needs fine tuning. Can look at different selection. 
Cons: Some routes are horrendous. 

Experiment: Tried improving the crossover, by picking the route out of the two parents, with the closest distance to the target_node s the tail, got stuck

Improvement -> Including a weighting/modifying the fitness. Population generation would mainly need improving, as if this is initially bad, then so are the rest of the routes.

In [5]:
import GA
coords = GA.generate_route()
folium.PolyLine(
    locations=coords,
    color="#FF0000",
    weight=5,
    tooltip="P2P-GA",
    opacity=0.5,
).add_to(m)

c:\Users\Dan-L\scoop\apps\miniconda3\24.7.1-0\Lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Generating Population
Heuristic: {18085463: 0.12418457343909556, 18085464: 0.12146208884923727, 18086229: 0.09675352235102351, 18086251: 0.03972698807511082, 18086252: 0.03710858340936763, 18086260: 0.02995709846630657, 18086264: 0.029361692154233728, 18086272: 0.03153781463735246, 18086290: 0.013455877532513967, 18086296: 0.005316877810332012, 18086300: 0.0015029491009347154, 18086301: 0.0033940740858147663, 18086303: 0.0035052984922823616, 18086305: 0.006496661238667148, 18086313: 0.009782848127718584, 18086329: 0.03642987432382861, 18086332: 0.03801259600408752, 18086338: 0.044239431034881024, 18086581: 0.12509995530039214, 18086599: 0.1230596185806706, 21232138: 0.04437249684770985, 21232141: 0.04407935540817165, 21232145: 0.04432242070758051, 21232150: 0.04207300246297878, 21232171: 0.030950135318765927, 21232197: 0.0532040055984501, 21232200: 0.05429899129081374, 21232202: 0.054651683664549394, 24940151: 0.05410184335972304, 26419947: 0.08604640421458551, 26420308: 0.064962538720

In [6]:
m

## Iterative Deepening Depth First Search

Regular DFS, but with a cap on the amount of nodes.

Time: (5653 Seconds from 1), (20 Seconds from 30) (10km)

Pros: Consistent results, although contain weird decisions
Cons: Can take time to build up to the 'first solution depth',  takes significant time if route is long (e.g 15km), as has to build up to the nodes

Improvement: Look into some sort of cache (saves the route, then extends them at a later stage)?

In [2]:
import iddfs
coords = iddfs.generate_route(10)
folium.PolyLine(
    locations=coords,
    color="#0000FF",
    weight=5,
    tooltip="P2P - IDDFS",
    opacity=0.5,
).add_to(m)
m

c:\Users\Dan-L\scoop\apps\miniconda3\24.7.1-0\Lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


6.279847621917725
Amount of Nodes: 1452
Amount of Edges: 3942
Generating Route
0.0
Depth: 1
Depth: 2
Depth: 3
Depth: 4
Depth: 5
Depth: 6
Depth: 7
Depth: 8
Depth: 9
Depth: 10
Depth: 11
Depth: 12
Depth: 13
Depth: 14
Depth: 15
Depth: 16
0.30172276496887207
Route Found
Distance: 9.789465
31


## Depth First Search

Regular DFS

Time: 30 seconds (10km)

Pros: Quicker than IDDFS
Con: Goes for greatest depth first, which may not always be optimal. Time increments 

In [11]:
import dfs
dfs_gen = dfs.DFS_Generator(10,1)

In [12]:
coords = dfs_gen.generate_route()
folium.PolyLine(
    locations=coords,
    color="#FF00FF",
    weight=5,
    tooltip="P2P - DFS",
    opacity=0.5,
).add_to(m)
m

0.10227084159851074
Route Found
Distance: 9.309891000000002


: 

In [5]:
m